In [1]:
import evaluate

In [2]:
rouge_test = evaluate.load('rouge')

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'kreimben/CodeMind-gemma'

tokenizer = AutoTokenizer.from_pretrained('kreimben/CodeMind-gemma')
tokenizer.padding_side = 'left'
model = AutoModelForCausalLM.from_pretrained('kreimben/CodeMind-gemma')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
%%time

from utils import get_completion

import torch

with torch.no_grad():
    res = get_completion('hello!', model, tokenizer, device='cpu', max_new_tokens=32).split('model')[-1]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


CPU times: user 1min 34s, sys: 1min 41s, total: 3min 16s
Wall time: 1min 35s


In [15]:
res

'hello! greetings to you as well. how can i be of service today? do you need to discuss a project, complete a task, or have a brief'